In [1]:
import json
import os
from getpass import getpass
import psutil
IN_NOTEBOOK = any(["jupyter-notebook" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

Secrets (JSON string):  ········


In [2]:
import openai
import os
import autogen

In [3]:
os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v2']['credential'] 
os.environ["TOGETHERAI_API_KEY"] = CREDS['together-ai']['key']['credential']
os.environ['ANTHROPIC_API_KEY'] = CREDS['anthropic']['key']['credential']
#openai.api_key = CREDS['OpenAI']['v2']['credential'] 

# Get list of available OpenAI models

In [4]:
client = openai.OpenAI()
model_list = client.models.list()
for model in model_list:
  print(model.id)

dall-e-3
gpt-4-1106-preview
whisper-1
gpt-4-0613
davinci-002
gpt-4
babbage-002
dall-e-2
gpt-3.5-turbo-16k
tts-1-hd-1106
tts-1-hd
gpt-3.5-turbo-1106
gpt-4o-2024-05-13
gpt-4o
gpt-4-turbo-2024-04-09
gpt-3.5-turbo-instruct-0914
gpt-4-turbo
gpt-3.5-turbo-instruct
gpt-4-0125-preview
text-embedding-3-small
gpt-4-turbo-preview
tts-1
gpt-3.5-turbo-0125
gpt-3.5-turbo
gpt-3.5-turbo-0301
tts-1-1106
text-embedding-3-large
gpt-3.5-turbo-0613
text-embedding-ada-002
gpt-4-1106-vision-preview
gpt-3.5-turbo-16k-0613
gpt-4-vision-preview


In [5]:
config_list = [{"model": "gpt-4-1106-preview", "api_key": os.getenv("OPENAI_API_KEY")}]

# First Test

In [12]:
from autogen import ConversableAgent

cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

johann = ConversableAgent(
    "johann",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

result = cathy.initiate_chat(johann, message="Johann, tell me a joke.", max_turns=2)

cathy (to johann):

Johann, tell me a joke.

--------------------------------------------------------------------------------
johann (to cathy):

Why don't scientists trust atoms?

Because they make up everything!

--------------------------------------------------------------------------------
cathy (to johann):

Oh, Johann! You really have a knack for those science jokes, don't you? Let's see if I can match that.

Why don't we ever tell secrets on a farm?

Because the potatoes have eyes, the corn has ears, and the beans stalk!

--------------------------------------------------------------------------------
johann (to cathy):

Hahaha, Joe! That's a good one! You always know how to keep the crowd laughing. Let's see, here's another one:

Why don't some fish play piano?

Because you can't tuna fish!

--------------------------------------------------------------------------------


# Modify software in filesystem

Source: https://microsoft.github.io/autogen/docs/notebooks/agentchat_function_call_code_writing/

As the scope of possible operations is predefined inside the tools, we can safely use Autogen **without Docker**, avoiding all the complications related to it.

In [39]:
llm_config = {
    "temperature": 0,
    "config_list": config_list,
}

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message="""
    I'm Engineer. I'm expert in python programming. I'm executing code tasks required by Admin.
    """,
)

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    human_input_mode="ALWAYS",
    #human_input_mode="NEVER",
    code_execution_config=False,
)

In [40]:
groupchat = autogen.GroupChat(
    agents=[engineer, user_proxy],
    messages=[],
    max_round=500,
    speaker_selection_method="round_robin",
    enable_clear_history=True,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [41]:
from typing_extensions import Annotated

default_path = "./backend_dir/"


@user_proxy.register_for_execution()
@engineer.register_for_llm(description="List files in choosen directory.")
def list_dir(directory: Annotated[str, "Directory to check."]):
    files = os.listdir(default_path + directory)
    return 0, files


@user_proxy.register_for_execution()
@engineer.register_for_llm(description="Check the contents of a chosen file.")
def see_file(filename: Annotated[str, "Name and path of file to check."]):
    with open(default_path + filename, "r") as file:
        lines = file.readlines()
    formatted_lines = [f"{i+1}:{line}" for i, line in enumerate(lines)]
    file_contents = "".join(formatted_lines)

    return 0, file_contents


@user_proxy.register_for_execution()
@engineer.register_for_llm(description="Replace old piece of code with new one. Proper indentation is important.")
def modify_code(
    filename: Annotated[str, "Name and path of file to change."],
    start_line: Annotated[int, "Start line number to replace with new code."],
    end_line: Annotated[int, "End line number to replace with new code."],
    new_code: Annotated[str, "New piece of code to replace old code with. Remember about providing indents."],
):
    with open(default_path + filename, "r+") as file:
        file_contents = file.readlines()
        file_contents[start_line - 1 : end_line] = [new_code + "\n"]
        file.seek(0)
        file.truncate()
        file.write("".join(file_contents))
    return 0, "Code modified"


@user_proxy.register_for_execution()
@engineer.register_for_llm(description="Create a new file with code.")
def create_file_with_code(
    filename: Annotated[str, "Name and path of file to create."], code: Annotated[str, "Code to write in the file."]
):
    with open(default_path + filename, "w") as file:
        file.write(code)
    return 0, "File created successfully"

The return type of the function 'list_dir' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.
The return type of the function 'see_file' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.
The return type of the function 'modify_code' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.
The return type of the function 'create_file_with_code' is not annotated. Although annotating it is optional, the function should return either a string, a subclass of 'pydantic.BaseModel'.


# Create file that agents should modify

Clear directory before

In [42]:
import shutil
shutil.rmtree('backend_dir')
os.mkdir('backend_dir')
os.mkdir('backend_dir/app')

In [43]:
%%writefile backend_dir/app/main.py

from fastapi import FastAPI
import yfinance as yf

app = FastAPI()

@app.get("/cdr_daily_spread")
async def calculate_daily_spread():
    cdr = yf.Ticker("CDR.WA")
    today_data = cdr.history(period="1d")
    spread = ((today_data["High"] - today_data["Low"]) / today_data["Low"]) * 100
    return spread

Writing backend_dir/app/main.py


In [ ]:
chat_result = user_proxy.initiate_chat(
    manager,
    message="""
        You will need to improve app in FastApi. 
        For now, check out all the application files, try to understand it and wait for next instructions.""",
)

Admin (to chat_manager):


        You will need to improve app in FastApi. 
        For now, check out all the application files, try to understand it and wait for next instructions.

--------------------------------------------------------------------------------
Engineer (to chat_manager):

***** Suggested tool call (call_SgwUUwxwmWFF88WIgbpeAmZ0): list_dir *****
Arguments: 
{"directory":"./app"}
*************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION list_dir...
Admin (to chat_manager):

Admin (to chat_manager):

***** Response from calling tool (call_SgwUUwxwmWFF88WIgbpeAmZ0) *****
[0, ["main.py"]]
**********************************************************************

--------------------------------------------------------------------------------
Engineer (to chat_manager):

***** Suggested tool call (call_cj5gYfs7Z4znObapgBB19IMW): see_file *****
Argu